In [1]:
# Import các thư viện cần thiết
import pandas as pd
import numpy as np
import os
from pathlib import Path


current_path = Path(os.getcwd())

PROJECT_ROOT = current_path.parent 

CITY = 'Paris'
# Các ngày snapshot của Paris
SNAPSHOT_DATES = ['06 December, 2024', '03 March, 2025', '06 June, 2025', '12 September, 2025']

RAW_DIR = PROJECT_ROOT / 'raw' / CITY
PROCESSED_DIR = PROJECT_ROOT / 'processed' / CITY
REPORTS_DIR = PROJECT_ROOT / 'reports'
FIGURES_DIR = PROJECT_ROOT / 'figures'

# Tạo thư mục cần thiết
os.makedirs(PROCESSED_DIR, exist_ok=True)
os.makedirs(REPORTS_DIR, exist_ok=True)
os.makedirs(FIGURES_DIR, exist_ok=True)

print(f"Thư mục gốc được xác định: {PROJECT_ROOT}")

# Hàm Helper: Đọc file CSV 
def load_data(date_snapshot_raw, filename):
    """Tải file CSV (Giả định không nén) từ thư mục raw.
    
    Sử dụng TRỰC TIẾP TÊN SNAPSHOT GỐC (bao gồm dấu phẩy) để tạo đường dẫn,
    vì tên thư mục thực tế trên ổ đĩa của bạn có chứa dấu phẩy.
    """
    
    # --- KHÔNG CHUẨN HOÁ TÊN THƯ MỤC NỮA ---
    folder_name = date_snapshot_raw 
    
    file_path = RAW_DIR / folder_name / filename
    
    if not file_path.exists():
        print(f"File {filename} không tồn tại tại {file_path}")
        return None
    return pd.read_csv(file_path, low_memory=False)

# Hàm Helper: Làm sạch tiền tệ/tỷ lệ
def clean_currency_to_float(series):
    """Làm sạch các giá trị tiền tệ/tỷ lệ từ chuỗi sang float."""
    if series.dtype == 'object':
        series = series.astype(str).str.replace(r'[$,%]', '', regex=True)
    return pd.to_numeric(series, errors='coerce')

Thư mục gốc được xác định: d:\btlltxldl\2526-LTXLDL-Project4-AIT2006-2-4.4


In [5]:
# Danh sách để thu thập các vi phạm QA
qa_summary_list = []

def process_listings(df_listings, date_snapshot):
    """Làm sạch, tạo đặc trưng và gắn cờ QA cho listings."""
    df_cleaned = df_listings.copy()
    
    # 1. Chuẩn hoá và Làm sạch
    df_cleaned['price'] = clean_currency_to_float(df_cleaned['price'])
    rate_cols = ['host_response_rate', 'host_acceptance_rate']
    for col in rate_cols:
        if col in df_cleaned.columns:
            df_cleaned[col] = clean_currency_to_float(df_cleaned[col]) / 100
    
    df_cleaned['last_scraped'] = pd.to_datetime(df_cleaned['last_scraped'])
    df_cleaned['host_since'] = pd.to_datetime(df_cleaned['host_since'])
    
    # 2. Gắn cờ QA
    df_cleaned['flag_price_outlier'] = (df_cleaned['price'].isna()) | (df_cleaned['price'] <= 0) | (df_cleaned['price'] > 5000)
    violated_count = df_cleaned['flag_price_outlier'].sum()
    if violated_count > 0:
        qa_summary_list.append({
            'snapshot_date': date_snapshot,
            'rule_id': 'flag_price_outlier', # Tên rule ngắn gọn
            'records_affected': violated_count,
            'handling_decision': 'Gắn cờ'
        })
    
    # 3. Tạo Đặc trưng
    df_cleaned['host_age_days'] = (df_cleaned['last_scraped'] - df_cleaned['host_since']).dt.days
    df_cleaned['price_per_person'] = np.where(
        df_cleaned['accommodates'] > 0,
        df_cleaned['price'] / df_cleaned['accommodates'],
        np.nan
    )
    
    # THÊM CỘT SNAPSHOT_DATE VÀO ĐÂY
    df_cleaned['snapshot_date'] = date_snapshot
    
    # Lọc các cột cần thiết cho bước tiếp theo
    core_cols = [col for col in df_cleaned.columns if col in df_listings.columns 
                 or col in ['host_age_days', 'price_per_person', 'flag_price_outlier', 'snapshot_date']] 
                 # ĐÃ THÊM 'snapshot_date' Ở CUỐI
                 
    return df_cleaned[core_cols]


def process_calendar(df_calendar, date_snapshot):
    """Làm sạch calendar (price) và tính toán cột cần thiết."""
    df_cal = df_calendar.copy()
    df_cal['clean_price'] = clean_currency_to_float(df_cal['price'])
    df_cal['is_booked'] = df_cal['available'].map({'f': 1, 't': 0})
    
    return df_cal[['listing_id', 'date', 'available', 'clean_price', 'is_booked']]


def process_reviews(df_reviews, date_snapshot):
    """Làm sạch reviews."""
    df_rev = df_reviews.copy()
    df_rev['date'] = pd.to_datetime(df_rev['date'], errors='coerce')
    
    # Lọc các cột cần thiết
    return df_rev[['listing_id', 'id', 'date']] # id trong reviews là review_id


def process_neighbourhoods(df_neighbourhoods, date_snapshot):
    """Xử lý neighbourhoods (chủ yếu là đọc và giữ nguyên, không cần làm sạch phức tạp)."""
    return df_neighbourhoods.copy()

In [6]:
FILES_TO_PROCESS = {
    'listings.csv': process_listings, 
    'calendar.csv': process_calendar, 
    'reviews.csv': process_reviews,
    'neighbourhoods.csv': process_neighbourhoods
}

print("--- BẮT ĐẦU LÀM SẠCH VÀ CHUẨN HOÁ (Mục 3) ---")

for date_snapshot in SNAPSHOT_DATES:
    print(f"\n[SNAPSHOT] Đang xử lý: {date_snapshot}")
    
    # Tạo thư mục đầu ra trong processed/Paris/<date> (sử dụng tên snapshot có dấu phẩy)
    snapshot_output_dir = PROCESSED_DIR / date_snapshot
    os.makedirs(snapshot_output_dir, exist_ok=True)
    
    for filename, processor_func in FILES_TO_PROCESS.items():
        # 1. Tải dữ liệu
        df_raw = load_data(date_snapshot, filename)
        if df_raw is None:
            continue
            
        # 2. Xử lý
        df_processed = processor_func(df_raw, date_snapshot)
        
        # 3. Lưu file *_processed.csv (theo cấu trúc yêu cầu)
        output_filename = filename.replace('.csv', '_processed.csv')
        output_path = snapshot_output_dir / output_filename
        df_processed.to_csv(output_path, index=False)
        
        print(f"  ✅ Đã xử lý và lưu: {output_filename} ({len(df_processed):,} hàng)")

# 4. Tổng hợp và Lưu Báo cáo QA (Mục 3)
if qa_summary_list:
    qa_df = pd.DataFrame(qa_summary_list)
    qa_output_path = REPORTS_DIR / 'Paris_qa_summary.csv'
    qa_df.to_csv(qa_output_path, index=False)
    
    print("\n--- BÁO CÁO QA TỔNG HỢP ---")
    print(f"\n✅ Đã lưu Báo cáo QA tổng hợp tại: {qa_output_path}")
else:
    print("\nKhông có vi phạm QA nào được ghi nhận.")

print("\n--- HOÀN TẤT NOTEBOOK PARIS_EDA.ipynb ---")

--- BẮT ĐẦU LÀM SẠCH VÀ CHUẨN HOÁ (Mục 3) ---

[SNAPSHOT] Đang xử lý: 06 December, 2024
  ✅ Đã xử lý và lưu: listings_processed.csv (91,031 hàng)
  ✅ Đã xử lý và lưu: calendar_processed.csv (33,226,170 hàng)
  ✅ Đã xử lý và lưu: reviews_processed.csv (2,063,010 hàng)
  ✅ Đã xử lý và lưu: neighbourhoods_processed.csv (20 hàng)

[SNAPSHOT] Đang xử lý: 03 March, 2025
  ✅ Đã xử lý và lưu: listings_processed.csv (86,064 hàng)
  ✅ Đã xử lý và lưu: calendar_processed.csv (31,442,697 hàng)
  ✅ Đã xử lý và lưu: reviews_processed.csv (2,068,800 hàng)
  ✅ Đã xử lý và lưu: neighbourhoods_processed.csv (20 hàng)

[SNAPSHOT] Đang xử lý: 06 June, 2025
  ✅ Đã xử lý và lưu: listings_processed.csv (84,055 hàng)
  ✅ Đã xử lý và lưu: calendar_processed.csv (0 hàng)
  ✅ Đã xử lý và lưu: reviews_processed.csv (2,173,219 hàng)
  ✅ Đã xử lý và lưu: neighbourhoods_processed.csv (20 hàng)

[SNAPSHOT] Đang xử lý: 12 September, 2025
  ✅ Đã xử lý và lưu: listings_processed.csv (81,853 hàng)
  ✅ Đã xử lý và lưu: ca